In [2]:
import folium
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_excel('ca.xlsx')
df.head(10)

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Toronto,43.666667,-79.416667,Canada,CA,Ontario,admin,5213000,3934421
1,Montréal,45.500000,-73.583333,Canada,CA,Québec,NaN,3678000,2356556
2,Vancouver,49.250000,-123.133333,Canada,CA,British Columbia,NaN,2313328,603502
3,Ottawa,45.416667,-75.700000,Canada,CA,Ontario,primary,1145000,812129
4,Calgary,51.083333,-114.083333,Canada,CA,Alberta,NaN,1110000,915322
5,Edmonton,53.550000,-113.500000,Canada,CA,Alberta,admin,1058000,712391
6,Hamilton,43.256101,-79.857484,Canada,CA,Ontario,NaN,721053,519949
7,Winnipeg,49.883333,-97.166667,Canada,CA,Manitoba,admin,632063,575313
8,Québec,46.800000,-71.250000,Canada,CA,Québec,admin,624177,528595
9,Oshawa,43.900000,-78.866667,Canada,CA,Ontario,NaN,450963,247989
